In [ ]:
"""
Create a function that maps the labels to the synapses associated with them

Will be a little different than pinky100 version:


"""


In [25]:
import datajoint as dj
import numpy as np

import datajoint as dj
import numpy as np
import pykdtree.kdtree as KDTree
import time
from tqdm import tqdm


In [20]:
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")
pinky = dj.create_virtual_module("pinky","microns_pinky")

In [13]:
dj.U("segment_id","version") & m65.ProofreadLabel()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron
96631955273149705,0
107816118160698192,0


In [12]:
# m65.WholeAutoAnnotationsLabelClusters3.describe()
# m65.ProofreadLabel.describe()

-> m65.FromNeuromancer
decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
author_original      : varchar(20)                  # name of last editor
author_proofreader   : varchar(20)                  # name of last editor
date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
---
vertices             : longblob                     # label data for the vertices
triangles            : longblob                     # label data for the faces
edges                : longblob                     # label data for the edges
status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey

-> m65.FromNeuromancer
decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
author_original      : varchar(20)                  # name of last editor
author_proofreader   : varchar(20)                  # name of last editor
date_ti

'-> m65.FromNeuromancer\ndecimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated\nauthor_original      : varchar(20)                  # name of last editor\nauthor_proofreader   : varchar(20)                  # name of last editor\ndate_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited\n---\nvertices             : longblob                     # label data for the vertices\ntriangles            : longblob                     # label data for the faces\nedges                : longblob                     # label data for the edges\nstatus               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey\n'

In [10]:
m65.FromNeuromancer()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,neuron_name Name of the neuron as it appears in the Neuromancer-Ingest website.,n_vertices,n_faces,vertices,faces
96631955273149705,0,RONNY,1659719,3161800,=BLOB=,=BLOB=
106626583548494129,0,WILSON,2938478,5601980,=BLOB=,=BLOB=
107816118160698192,0,CHARMAINE,937779,1791632,=BLOB=,=BLOB=


In [9]:
m65.Authors() & "username='spapadop'"

username username the person pcisk,real_name the real name of that corresponds to the username
spapadop,stelios papadopoulos


In [18]:
@schema
class SynapseCompartmentLabel(dj.Manual):
    definition = """
    -> m65.Synapse
    -> m65.FromNeuromancer
    decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
    ---
    (postsynaptic_label)->m65.LabelKey
    """ # (postsynaptic_label)->pinky.LabelKey : tinyint unsigned

    

In [48]:
#combine all the meshes into one mesh
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

In [38]:
subsegment_ids

In [40]:
subsegment_ids = list((m65.FromNeuromancer.Subsegment & lookup_key).fetch("subsegment_id"))
subsegment_ids.append(segment_id)
m65.Synapse() & [dict(postsyn=k) for k in subsegment_ids]

synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),size (EM voxels)
311550674,108102884604056386,107608035651997587,317442,216280,22094,4088
178940560,109786236570891677,107816118160698192,334724,168456,19596,1081
179010627,108661642467826005,107816118160698192,332958,168456,19820,2079
179261578,108941811707797312,107816118160698192,333458,168292,19727,402
179264051,109011080940145061,107816118160698192,334170,168062,19709,967
179682551,109434324130262546,107816118160698192,335074,169750,19478,642
179683895,107463930775139104,107816118160698192,335468,169879,19459,272
179684715,107463930775139104,107816118160698192,335648,170076,19438,945
179842333,106055593684964412,107816118160698192,310730,161298,19966,5164
179842460,105561913098018022,107816118160698192,310751,161168,19770,3169


In [60]:
# Practicing how to get the mesh and the full subsegments list

segment_id = 107816118160698192
version = 0
decimation_ratio = 1.00
author = "spapadop"

key = dict(segment_id=segment_id, version = version)
n = (m65.FromNeuromancer & key).fetch1()

#get all of the segments and their data=
subsegments = (m65.FromNeuromancer.Subsegment & key).fetch(as_dict=True)
subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]

# creating the entire mesh from the main mesh and all of its sub meshes: 
new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
                   main_mesh_faces=n["faces"],
                   sub_mesh_vertices = subsegments_vertices,
                   sub_mesh_faces=subsegments_faces)

#get the segment labels


#find the synapses for neuron and all subsegments
subsegment_ids = list((m65.FromNeuromancer.Subsegment & lookup_key).fetch("subsegment_id"))
subsegment_ids.append(segment_id)
segment_synapses = np.array((m65.Synapse() & [dict(presyn=k) for k in subsegment_ids]).fetch('synapse_id', 'centroid_x', 'centroid_y', 'centroid_z')).T

#
labels_search_key = dict(key,author=author,
                         status='complete',
                         decimation_ratio=decimation_ratio
                        )

segment_vert_labels, segment_tri_labels = (m65.ProofreadLabel() & labels_search_key).fetch("vertices","triangles")
len(segment_vert_labels)

if len(segment_vert_labels) <= 0:
    print(f"Skipping the mesh {segment_id} because it has no labels")


Added 46 subsegements 
  --> final mesh: 1252162 vertices and 2390205 faces
Skipping the mesh 107816118160698192 because it has no labels


In [ ]:
version = 0
decimation_ratio = 1.00
author = "celiib"

#get all the neurons for that user that have complete status (aka have been proofread)

In [ ]:
#for the regular excitatory meshes
segment_ids = pinky.CoarseLabelFinal.fetch('segment_id') # get all the segment ids
mesh_fetch = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh & pinky.CompartmentFinal.proj()).fetch()
label_fetch = pinky.CoarseLabelFinal.fetch()

meshes = dict()
labels = dict()
#maps the segment id to the mesh vertices and triangles array
for segment in tqdm(mesh_fetch):
    meshes[segment[1]] = (segment[5], segment[6]) #maps the id number to the vertices and triangles

#maps segment_id to the mesh vertices and triangle labels
for segment in tqdm(label_fetch):
    labels[segment[1]] = (segment[6], segment[7]) #maps the id number to the vertices and triangle labels

#builds a KD tree fro mthe mesh vertices
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0]) #creates a KDTree from the mesh vertices

#maps segment_id to the mesh synapses and coordinates (this needs to include all the subsegements)
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
#                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'presyn_x', 'presyn_y', 'presyn_z')).T
                                           & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x', 'postsyn_y', 'postsyn_z')).T

In [ ]:
matched_synapses = dict()
synapses_to_exclude = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude.append(dict(segmentation=2,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    matched_synapses[segment_id] = synapse_labels

In [ ]:
#add to synapse exclude
ta3p100.SynapseExcludeFinal.insert(synapses_to_exclude,skip_duplicates=True)

In [ ]:
segmentation = ta3p100.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>12] = 1

start = time.time()
SynapseCompartmentLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)